### Description

Here should be exploratorial analysis - different plots, attempts to understand data, etc.

In [16]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.listdir("../data")

['.DS_Store',
 'description.txt',
 'input_wagon.xlsx',
 'output_wagon.xlsx',
 'wagon.zip']

In [3]:
df = pd.read_excel("../data/input_wagon.xlsx", sheet_name=None)

In [4]:
list(df)

['Orders_07',
 'Orders_08',
 'WagonModelCompatibility',
 'sources',
 'standplaces',
 'Metrics']

# EDA таблиц

## 1. Orders_07

Что сделать/выяснить:
- [ ] посмотреть ID заказа - есть ли закономерности. Часть номеров короткие - с чем это может быть связано?
- [ ] тариф: как интерпретировать набор цифр с разделителями?

In [5]:
def read_table(table_name="Orders_07"):
    table_df = df[table_name]
    columns = table_df.columns
    column_names = table_df.iloc[0].values
    table_df = table_df.loc[1:]
    table_df.columns = column_names
    for c, d in zip(columns, column_names):
        print(d, "-", c)
    return table_df

orders7 = read_table("Orders_07")

OrderNum - ID заказа
Start - Вершина старта
Finish - Вершина финиша
StartDate - Дата старта относительно дня запуска расчета, сутки
Dur - Время выполнения заказа (движения по арке+погрузка и разгрузка, сутки)
MinUnit - Минимальный обьем заказа, вагоны
MaxUnit - Максимальный обьем заказа, вагоны
Tariff - Тариф в зависимотси от отклонения от даты старта ( за вагон), рубли
NeedWagonModel - Запрошенная категория вагона
ShortagePenalty - Категория приоритетности заказа (убывание от 1 до 4)


In [18]:
orders7.head()

,OrderNum,Start,Finish,StartDate,Dur,MinUnit,MaxUnit,Tariff,NeedWagonModel,ShortagePenalty
1,302201_812941937003_0707,932103,937003,0,13,3,3,53514:52334:51154:49974,2039,4
2,303139_830582944702_0707,946801,944702,0,13,2,3,8550:7370:6190:5010,2025,4
3,303933_830578944702_0707,946801,944702,0,13,2,3,8550:7370:6190:5010,2025,4
4,305170_830580944702_0707,946801,944702,0,13,2,3,8550:7370:6190:5010,2025,4
5,303071_814704942105_0707,946801,942105,0,14,2,2,25768:24588:23408:22228,2011,4


In [7]:
start_nodes = orders7.Start
end_nodes = orders7.Finish
nodes = np.concatenate([
    np.unique(start_nodes),
    np.unique(end_nodes)
])
nodes = np.unique(nodes)
nodes.shape

(709,)

## Orders_8

In [8]:
orders8 = read_table("Orders_08")

OrderNum - ID заказа
Start - Вершина старта
Finish - Вершина финиша
StartDate - Дата старта относительно дня запуска расчета, сутки
Dur - Время выполнения заказа (движения по арке+погрузка и разгрузка, сутки)
MinUnit - Минимальный обьем заказа, вагоны
MaxUnit - Максимальный обьем заказа, вагоны
Tariff - Тариф в зависимотси от отклонения от даты старта ( за вагон), рубли
NeedWagonModel - Запрошенная категория вагона
ShortagePenalty - Категория приоритетности заказа (убывание от 1 до 4)


In [9]:
start_nodes = orders8.Start
end_nodes = orders8.Finish
nodes = np.concatenate([
    np.unique(start_nodes),
    np.unique(end_nodes)
])
nodes = np.unique(nodes)
nodes.shape

(648,)

## WagonModelCompatibility

Вопросы:
- [ ] совместимы ли разные типы вагонов в одну сторону, или связь двусторонняя? То есть если A -> B (A - запрошенный тип, B - совместимый тип), C->B, можно ли сказать, что A->C, а C -> A?

In [10]:
wagon_mode_compat = read_table("WagonModelCompatibility")

NeedWagonModel - Запрошенный тип вагона
CompatibleWagonModel - Совместимый тип вагона


In [11]:
w1 = np.unique(wagon_mode_compat.iloc[:, 0])
w2 = np.unique(wagon_mode_compat.iloc[:, 1])
w3 = np.unique(np.concatenate([w1, w2]))
w1.shape, w2.shape, w3.shape

((492,), (63,), (493,))

## sources

In [12]:
sources = read_table("sources")

Station - Станция дислокации (стоянки или освобождения после заказа)
Date - День, начиная с которого вагоны доступны (отсчет с дня запуска расчета)
Units - Количество вагонов, в  день дислокации
Stand - Формула расчета простоя (руб за сутки простоя на станции дислокации 1 вагона)
WagonModel - Модель вагонов


## standplaces

In [13]:
standplaces = read_table("standplaces")

Station - Код станции отстоя
PricePut - Стоимость отстоя, руб за факт входа без расчета суток
PriceStand - Цена простоя в сутки


## Metrics

In [14]:
metrics = read_table("Metrics")

From - Узел старта
To - Узел финиша
Group - Группа пробега
Distance - Расстояние пробега, км
Time - Время пробега, сутки
PriceUnit - Стоимость пробега, руб


In [17]:
metrics.head()

,From,To,Group,Distance,Time,PriceUnit
1,10002,10303,100,9,5,104
2,10002,10303,200,9,5,138
3,10002,10303,305,9,5,355
4,10002,10905,100,60,5,685
5,10002,10905,200,60,5,914
